In [1]:
# !pip install pandas numpy scikit-learn transformers
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import re
import torch

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

from transformers import BertTokenizer, BertModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
job_df = pd.read_parquet('/kaggle/input/job-offers/reed_uk_data.parquet')[['job_title', 'job_description']].sample(1000)
resume_df = pd.read_csv('/kaggle/input/resumecorpus-cleaned/finale.csv')[['Label', 'Text']].sample(1000)

print(f"job data shape: {job_df.shape}, resume: {resume_df.shape}")

job data shape: (1000, 2), resume: (1000, 2)


In [3]:
job_df.sample(3)

,job_title,job_description
43019,Urgent - CRM/Membership Coordinator needed - u...,Apply now Are you looking for a role in which...
46791,LENDING OFFICER (CANTONESE) - INTERNATIONAL BA...,Apply now Â£35K + BENS + BONUS An exciting op...
38106,Finance Business Partner,Apply now Finance Business Partner - 12/14 mo...


In [4]:
resume_df.sample(3)

,Label,Text
14386,"[b'Java_Developer\n', b'Software_Developer']",Java Software DeveloperStudent Programmer Inte...
24079,[b'Systems_Administrator'],Systems Administrator IService Desk Technician...
7078,[b'Project_manager'],Senior Program ManagerIT Project ManagerIT Pro...


In [6]:
# !pip install skillner

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.7 MB/s eta 0:00:0000:01
  Created wheel for skillner: filename=skillNer-1.0.3-py3-none-any.whl size=25648 sha256=34e7279bb04eb8955893dfa39b44e0eee58ff4b9f8a00a3203237bcffdfede8b
  Stored in directory: /root/.cache/pip/wheels/9a/f1/7a/3201e01141cdb862654585eb16412785ed9973459639c351e4
Successfully built skillner


In [8]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [9]:
from transformers import pipeline

# Load the BERT-based model for summarization
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import subprocess
import torch.nn as nn

In [11]:
# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
    nltk.data.find('stopwords.zip')
    nltk.data.find('punkt.zip')
except:
    nltk.download('punkt', download_dir='/kaggle/working/')
    nltk.download('stopwords', download_dir='/kaggle/working/')
    nltk.download('wordnet', download_dir='/kaggle/working/')
    
    command = "unzip /kaggle/working/corpora/punkt.zip -d /kaggle/working/corpora"
    command = "unzip /kaggle/working/corpora/stopwords.zip -d /kaggle/working/corpora"
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"

    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet, stopwords

[nltk_data] Downloading package punkt to /kaggle/working/...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /kaggle/working/...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  


In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # remove special characters and digits
    text = re.sub(r'\W+', ' ', text.lower())
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stop words and perform lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    # rejoin the tokens into a string
    clean_text = ' '.join(tokens)
    return clean_text


In [13]:
job_df['clean_description'] = list(tqdm(job_df['job_description'].apply(preprocess_text), desc='cleaning offers desc'))
resume_df['clean_content'] = list(tqdm(resume_df['Text'].apply(preprocess_text), desc='cleaning resume cont'))

cleaning offers desc:   0%|          | 0/1000 [00:00<?, ?it/s]

cleaning resume cont:   0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
def summarize_text(text):
    # Summarize the text if it exceeds 250 words
    if len(text.split()) > 250:
        summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
        if summary:
            return summary[0]["summary_text"]
        else:
            return text
    else:
        return text

In [19]:
job_df['clean_description'] = list(tqdm(job_df['clean_description'].apply(summarize_text), desc='suumarizing offers desc'))
resume_df['clean_content'] = list(tqdm(resume_df['clean_content'].apply(summarize_text), desc='suumarizing resume cont'))

IndexError: index out of range in self

In [ ]:
# Annotate the summarized text using SkillNER
annotations = skill_extractor.annotate(summarized_text)